In [1]:
!pip install transformers accelerate #  모델의 GPU/TPU 가속화와 분산 학습을 도와주는 라이브러리

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")

model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

In [2]:
# save_path = "/content/drive/MyDrive/yaife/model_checkpoint"

# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

NameError: name 'model' is not defined

In [ ]:
# # 모델 로드
# from transformers import AutoTokenizer, AutoModelForCausalLM
# load_path = "/content/drive/MyDrive/yaife/model_checkpoint"

# tokenizer = AutoTokenizer.from_pretrained(load_path)
# model = AutoModelForCausalLM.from_pretrained(load_path)

In [3]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# test --1 with 유해요소 template
input_text = """
Input Text: 너 진짜 재수 없어. 한남 특징 다 갖췄네.
Harmful Spans: [('재수'), ('한남')]

input_text와 Harmful spans 정보를 가지고 상대가 기분이 나쁘지 않게 완화된 표현으로 표현해줘:
수정된 문장:"""

# Tokenizer로 입력 데이터 준비
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    return_attention_mask=True
)

# Model generation with attention mask and pad token id 설정
initial_output = model.generate(
    inputs['input_ids'],  # 입력 ID
    attention_mask=inputs['attention_mask'],  # attention mask 추가
    max_new_tokens=500,
    temperature=0.5,
    top_k=10,
    top_p=0.8,
    pad_token_id=tokenizer.eos_token_id  # pad_token_id 설정
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)

#추가된 코드 1203
# 결과 파싱하여 수정된 문장 추출
def extract_modified_sentence(result):
    # "수정된 문장:" 이후 문장 추출
    start = result.find("수정된 문장:") + len("수정된 문장:")
    end = result.find("\n", start)
    return result[start:end].strip()

# 최종 수정된 문장 정리
modified_sentence = extract_modified_sentence(initial_result)

# 최종 출력
print("최종 수정된 문장:", modified_sentence)


In [6]:
# test 0
# test 1- with 유해요소 template
input_text = """
Input Text: 그 때 처빨았던 과거의 행적이 욕을 먹는 건데 뭔 개소리하니
Harmful Spans: [('처', (4, 5)), ('##빨', (5, 6)), ('개소리', (28, 31))]

주어진 input_text에서 Harmful spans 정보를 가지고 유해한 표현을 제거하고 완화된 표현으로 재구성해줘:
수정된 문장:"""

# Tokenizer로 입력 데이터 준비
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    return_attention_mask=True
)

# Model generation with attention mask and pad token id 설정
initial_output = model.generate(
    inputs['input_ids'],  # 입력 ID
    attention_mask=inputs['attention_mask'],  # attention mask 추가
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id  # pad_token_id 설정
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)
print("Initial Result:", initial_result)


Initial Result: 
Input Text: 그 때 처빨았던 과거의 행적이 욕을 먹는 건데 뭔 개소리하니
Harmful Spans: [('처', (4, 5)), ('##빨', (5, 6)), ('개소리', (28, 31))]

주어진 input_text에서 Harmful spans 정보를 가지고 유해한 표현을 제거하고 완화된 표현으로 재구성해줘:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는 건데 뭔 개소리하니

정답은 다음과 같습니다:
수정된 문장: 그 때 과거의 행적이 욕을 먹는


In [5]:
# test 1- with 유해요소 template
input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

# Tokenizer로 입력 데이터 준비
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    return_attention_mask=True
)

# Model generation with attention mask and pad token id 설정
initial_output = model.generate(
    inputs['input_ids'],  # 입력 ID
    attention_mask=inputs['attention_mask'],  # attention mask 추가
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id  # pad_token_id 설정
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)
print("Initial Result:", initial_result)


Initial Result: 
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움을 받을 수 있는 사람이다.

문장에서 '구제불능'이라는 표현을 제거하고, 긍정적인 표현 '도움을 받을 수 있는 사람'으로 대체했습니다. 이는 긍정적인 의미를 강조하면서도, 단순히 '구제불능'이라는 단어만 제거하는 것보다는 더 깊이 있는 표현을 사용한 것입니다. 이로 인해 문장은 더 긍정적이고 긍정적인 인상을 줄 수 있습니다. 

또한, 이 표현은 단순히 단어를 교체하는 것만 아니라, 문장의 의미를 더 깊이 있게 표현하는 방식입니다. 이는 문장의 전달의도와 의미의 강도를 높이는 데 도움이 됩니다. 

이러한 변경은 문장의 전반적인 분위기를 개선하고, 더 긍정적인 인상을 주는 데 기여할 수 있습니다. 따라서, 문장의 긍정적인 표현을 강조하는 것은 중요하며, 이를 통해 문장의 의미를 더 깊이 있게 전달할 수 있습니다. 

또한, '


In [8]:
# test2- no 유해요소 template
input_text_without_template = """
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

without_output = model.generate(
    tokenizer(input_text_without_template, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

without_result = tokenizer.decode(without_output[0], skip_special_tokens=True)
print("without_result:", without_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


without_result: 
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 큰 도움을 주고 싶어한다.
기존 문장: 그들은 나아가게 될 것이라고 예상한다.
수정된 문장: 그들은 나아가게 될 것이라고 예상한다. (이 문장은 긍정적인 예측을 담고 있으므로 추가로 변경할 필요는 없고, 기존 문장이 충분하다.)

이러한 문장들은 주어진 문장에서 유해한 표현을 제거하고, 긍정적인 표현으로 변경하여 긍정적인 인상을 줄 수 있게 합니다. 이러한 작업은 글쓰기, 대화, 또는 일상생활에서 유용하게 사용될 수 있습니다. 

또한, 이러한 작업은 주어진 문장의 의미를 변경하지 않으면서도, 긍정적인 인상을 줄 수 있는 방법을 제공합니다. 예를 들어, "그는 정말 나쁜 사람이다"라는 문장을 "그는 정말 나쁜 사람이다"라는 문장을 "그는 정말 나쁜 사람이다"라는 문장을 "그는 정말 나쁜 사람이다"라는 문장을 "그는 정말 나쁜 사람이다"라는 문


In [12]:
# test 3- 더 길고 복잡한 문장으로 테스트 예제 1번
input_text_test3 = """
유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장:"""

test3_output = model.generate(
    tokenizer(input_text_test3, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.5,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id
)

test3_result = tokenizer.decode(test3_output[0], skip_special_tokens=True)
print("test3_Result:", test3_result)



test3_Result: 
유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장: 그는 매우 무식하지 않으며, 대화를 통해 자신의 생각을 표현하는 데 도움이 될 수 있다. 시간을 함께 보내면 인상적인 경험을 쌓을 수 있을 것 같아 기대된다. 그의 행동이 다른 사람에게 피해를 줄 수 있지만, 이를 통해 서로를 이해하고 도울 수 있는 기회가 될 수 있을 것이다. 그의 성格은 구체적이고, 그에게 도웘이 되는 사람들을 찾는 데 도움을 줄 수 있다.

이제 문제를 해결해 보겠습니다. 문제는 다음과 같습니다:
"유해한 요소가 있는 문장을 긍지하게 수정하세요."
문제를 해결하기 위해 다음과 같은 단계를 따릅니다:
1. 문장을 분석하고 유해 요소를 식별합니다.
2. 유해요소를 제거하여 긍성적인 표현으로 대체합니다.
3. 수정된 문장을 확인합니다.

이러한 단계들을 통해 문장을 유해를 제거한 후 긍정을 부여하는 글이 됩니다. 예를 들어, 다음과 같은 문장을 고려해 보세요:
"그 사람은 정말 아무 쓴도 없는 사람이 아니라고 생각한다."

이 문장을 수정


In [8]:
# Self-reflection
reflection_input = f"""
생성된 문장은 다음과 같습니다:
"{initial_result}"
이 문장은 원래 문장의 의미를 보존하며 유해요소를 완화하는 표현으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘. 완화되었고 문맥과 비슷하다면 생성을 멈춰줘.
"""

reflection_output = model.generate(
    tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

final_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
print("Final Result after Reflection:", final_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Final Result after Reflection: 
생성된 문장은 다음과 같습니다:
"
Input Text: 너 진짜 재수 없어. 한남 특징 다 갖췄네.
Harmful Spans: [('재수'), ('한남')]

주어진 input_text에서 Harmful spans 정보를 가지고 유해한 표현을 제거하고 완화된 표현으로 재구성해줘:
수정된 문장: 너 진짜 재수 없어. 한남 특징 다 갖췄네.
이전의 문장: 너 진짜 재수 없어. 한남 특징 다 갖췄네.

다음 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

이전 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

다음 문장으로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

이전 문장으로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

다음 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

이전 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

다음 문장으로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

이전 문장으로 이동:"
이 문장은 원래 문장의 의미를 보존하며 유해요소를 완화하는 표현으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘. 완화되었고 문맥과 비슷하다면 생성을 멈춰줘.
문맥에 맞지 않다면:
"너 진짜 재수 있어. 한남 특징 다 갖췄네."

다음 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

이전 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다 갖췄네.

다음 문장으로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다맞췄네.

이전 문장으로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다맞췄네.

다음 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다맞췄네.

이전 단어로 이동: 
1. 너 진짜 재수 있어.
2. 한남 특징 다맞췄네.

다음 문장으로 이동: 
1. 너

In [18]:
# 유사도를 이용한 재생성 여부 판단하기
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    """두 텍스트 간 유사도를 계산"""
    return SequenceMatcher(None, a, b).ratio()

def is_satisfactory(output):
    """모델의 출력이 적합한지 판단"""
    return "완화된 표현으로 변경되었습니다" in output or "완료" in output

prev_result = None
MAX_ITERATIONS = 3

for i in range(MAX_ITERATIONS):
    reflection_input = f"""
    이전 결과: "{prev_result if prev_result else "초기 입력"}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """

    inputs = tokenizer(initial_result, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)
    output = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=250,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )

    new_result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # 적합한 결과인지 확인
    if is_satisfactory(new_result):
        print(f"Iteration {i+1}: Satisfactory result found, stopping loop.")
        print("Final Result:", new_result)
        break

    # 유사도가 높은 경우 중단 (0.8이상)
    if prev_result and calculate_similarity(prev_result, new_result) > 0.8:
        print(f"Iteration {i+1}: No significant improvement, stopping loop.")
        print("Final Result:", new_result)
        break


    prev_result = new_result
    print(f"Iteration {i+1} Result:", new_result)


KeyboardInterrupt: 

## 평가모델


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

# Toxicity 평가 (KcELECTRA)
toxic_model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022")
toxic_tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")

# 유창성과 의미 평가 모델 (Sentence-BERT)
semantic_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def evaluate_text(original_text, generated_text):
    # 1. Toxicity 평가
    def get_toxicity_score(text):
        inputs = toxic_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = toxic_model(**inputs)
        score = torch.softmax(outputs.logits, dim=-1)[0][1].item()  # Toxic 클래스 점수
        return score

    original_toxicity = get_toxicity_score(original_text)
    generated_toxicity = get_toxicity_score(generated_text)
    toxicity_reduction = original_toxicity - generated_toxicity

    # 2. 유창성과 의미 평가
    def evaluate_fluency(text):
        inputs = toxic_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        perplexity = torch.exp(-inputs["input_ids"].size(1))  # Perplexity 계산 (단순 예제)
        return perplexity.item()

    def evaluate_similarity(original, generated):
        embedding1 = semantic_model.encode(original, convert_to_tensor=True)
        embedding2 = semantic_model.encode(generated, convert_to_tensor=True)
        similarity = util.cos_sim(embedding1, embedding2).item()
        return similarity

    fluency_score = evaluate_fluency(generated_text)
    semantic_similarity = evaluate_similarity(original_text, generated_text)

    # 유창성-일관성 점수
    fluency_consistency_score = 0.5 * fluency_score + 0.5 * semantic_similarity

    # 3. 최종 Critic 점수
    final_score = 0.7 * toxicity_reduction + 0.3 * fluency_consistency_score
    return {
        "original_toxicity": original_toxicity,
        "generated_toxicity": generated_toxicity,
        "toxicity_reduction": toxicity_reduction,
        "fluency_consistency_score": fluency_consistency_score,
        "final_score": final_score
    }

# 예제 테스트
original = "너는 정말 재수없어. 한남이야."
generated = "넌 그다지 매력적이지 않은 것 같아."

results = evaluate_text(original, generated)
print(results)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: exp(): argument 'input' (position 1) must be Tensor, not int